In [ ]:
import requests
from bs4 import BeautifulSoup

root_url = "https://www.immoweb.be/en/search/"
distribution_type = "BUY"
estate_type = "HOUSE"
# need to change in  a list 
# estate_type =['house', 'apartment'] 
country = "BE"

search_url = f"{root_url}{distribution_type}/{estate_type}?countries={country}"
print(search_url)
response = requests.get(search_url)
html_content = response.text
soup = BeautifulSoup(html_content, "html.parser")
house_listings = soup.find_all("a", class_="card__title-link")
house_urls = [listing["href"] for listing in house_listings]
for url in house_urls:
    print(url)


In [ ]:
import requests
from bs4 import BeautifulSoup
"house/for-sale?countries=BE&priceType=SALE_PRICE&page=1&orderBy=relevance"
"https://www.immoweb.be/en/search/house/for-sale?countries=BE&priceType=SALE_PRICE&page=333&orderBy=relevance"
root_url = "https://www.immoweb.be/en/search/"
estate_types = ['house', 'apartment']
max_page = 333  # Set the maximum page number to 333
immo_link = []
for estate in estate_types:
    page = 1
    while page <= max_page:
        url = f"{root_url}{estate}/for-sale?countries=BE&page={page}&orderBy=relevance"
        req = requests.get(url)
        print("Page: ", page)
        print("Status Code:", req.status_code)

        soup = BeautifulSoup(req.content, 'html.parser')
        card_results = soup.find_all('article', class_='card--result')

        href_links = []

        for article in card_results:
            link = article.find('a', class_='card__title-link')
            if link:
                href_links.append(link['href'])

        for i, link in enumerate(href_links, 1):
            immo_link = immo_link.append(link)
            print(i, link)


        page += 1

print("Scraping completed.")



In [ ]:
import re
import json
import requests
from bs4 import BeautifulSoup
import threading
from time import perf_counter

start_time = perf_counter()
root_url = "https://www.immoweb.be/en/search/"
estate_types = ['house', 'apartment']
max_page = 333  # Set the maximum page number to 333
immo_link = []

# Function to retrieve and parse data from a URL
def retrieve_data(url):
    response = requests.get(url) 
    js_content = response.text

    # Extract the content within the window.dataLayer assignment using regex
    match = re.search(r'window\.dataLayer\s*=\s*(\[.*?\]);', js_content)

    if match:
        data = match.group(1)  # Get the matched content within the first capturing group
        data_dict = json.loads(data)  # Parse the content into a Python dictionary
        
        # Replace empty values with 'none'
        def replace_empty_with_none(obj):
            for key, value in obj.items():
                if isinstance(value, dict):
                    replace_empty_with_none(value)
                elif isinstance(value, str) and not value:
                    obj[key] = 'none'
        
        replace_empty_with_none(data_dict)
        
        with lock:
            data_list.append(data_dict)

lock = threading.Lock() 
data_list = []

for estate in estate_types:
    page = 1
    while page <= max_page:
        url = f"{root_url}{estate}/for-sale?countries=BE&page={page}&orderBy=relevance"
        req = requests.get(url)
        print("Page:", page)
        print("Status Code:", req.status_code)

        soup = BeautifulSoup(req.content, 'html.parser')
        card_results = soup.find_all('article', class_='card--result')

        href_links = []

        for article in card_results:
            link = article.find('a', class_='card__title-link')
            if link:
                href_links.append(link['href'])

        for link in href_links:
            immo_link.append(link)
            # print("URL:", link)
            # Retrieve and parse the data from the URL using threading
            thread = threading.Thread(target=retrieve_data, args=(link,))
            thread.start()

        page += 1

print("Scraping completed.")
for thread in threading.enumerate():
    if thread != threading.current_thread():
        thread.join()

# Print the parsed data
# for index, data_dict in enumerate(data_list):
    # print(f"Data from URL {immo_link[index]}:")
    # print(data_dict)
    # print()

print(f"\nTime spent inside the loop: {perf_counter() - start_time} seconds.")    


In [18]:
import re
import json
import requests
from bs4 import BeautifulSoup
from time import perf_counter
from json import loads

start_time = perf_counter()
root_url = "https://www.immoweb.be/en/search/"
estate_types = ['house', 'apartment']
max_page = 333  # Set the maximum page number to 333
immo_link = []
data_dict =  []

url = "https://www.immoweb.be/en/classified/apartment-block/for-sale/henri-chapelle/4841/10130670"
# Function to retrieve and parse data from a URL
def retrieve_data(url):
        # print(url)
        response = requests.get(url)
        js_content = response.text
        # print(js_content)

        # Extract the content within the window.dataLayer assignment using regex
        match = re.search(r'window\.dataLayer\s*=\s*(\[.*?\]);', js_content)
        if match:
            data = match.group(1)  # Get the matched content within the first capturing group
            data_dict = json.loads(data)  # Parse the content into a Python dictionary

            soup = BeautifulSoup(requests.get(url).content)
            script_text = soup.find("script", text=re.compile("var\s+dataLayer")).text.split("= ", 1)[1]
            json_data = loads(script_text[:script_text.find(";")])
            print("json_data")
            print(json_data)

            # print(data_dict)
            # Replace empty values with 'none'
            def replace_empty_with_none(dict_to_clean):
                for key, value in dict_to_value.items():
                    if isinstance(value, dict):
                        replace_empty_with_none(value)
                    elif isinstance(value, str) and not value:
                        dict_to_clean[key] = 'none'
            replace_empty_with_none(data_dict)

            with lock:
                data_list.append(data_dict)
retrieve_data(url)
print(data_list)

[]


function :
    get_url_list
        list(url_list)

    get_immo_dict(url_list)
        list(immo_dict) or json
    
    replace_empty_with_none(immo_dict)
        list(cleaned_immo_dict) or json
        
    get_csv_file(immo_dict)
    
    